In [16]:
import pandas as pd
import numpy as np

def simulate_advanced_data(n=1000):
    np.random.seed(42)

    profiles = np.random.choice(["calm", "aggressive", "moderate"], size=n)

    df = pd.DataFrame({
        "daily_spending": np.random.normal(400, 120, n),
        "savings_progress": np.random.normal(0.6, 0.2, n),
        "debt_ratio": np.random.normal(0.3, 0.15, n),
        "bill_due_in_days": np.random.randint(1, 25, n),
        "profile": profiles
    })

    # Inject profile-based risky behaviors
    for i in range(n):
        if df.loc[i, "profile"] == "aggressive":
            if np.random.rand() > 0.7:
                df.loc[i, "daily_spending"] += np.random.uniform(400, 800)
                df.loc[i, "savings_progress"] -= np.random.uniform(0.1, 0.3)
                df.loc[i, "debt_ratio"] += np.random.uniform(0.2, 0.4)
        elif df.loc[i, "profile"] == "calm":
            if np.random.rand() > 0.9:
                df.loc[i, "daily_spending"] += np.random.uniform(200, 400)
                df.loc[i, "savings_progress"] -= np.random.uniform(0.05, 0.1)
    
    return df

df = simulate_advanced_data()
df.head()


,daily_spending,savings_progress,debt_ratio,bill_due_in_days,profile
0,405.822595,0.617681,0.097339,23,moderate
1,300.285986,0.895506,0.101631,24,calm
2,432.454819,0.371662,0.372280,22,moderate
3,393.971427,0.561268,0.382090,19,moderate
4,371.326234,0.456636,0.382333,18,calm


In [17]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Encode and scale
profile_enc = LabelEncoder()
df["profile_encoded"] = profile_enc.fit_transform(df["profile"])

features = ["daily_spending", "savings_progress", "debt_ratio", "bill_due_in_days", "profile_encoded"]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df[features])

model = IsolationForest(n_estimators=150, contamination=0.05, random_state=42)
model.fit(X_scaled)

df["anomaly"] = model.predict(X_scaled)
df["anomaly"] = df["anomaly"].map({1: 0, -1: 1})


In [18]:
def classify_behavior(row):
    if row["daily_spending"] > 700 and row["savings_progress"] < 0.3:
        return "Overspending & Under-saving"
    elif row["debt_ratio"] > 0.7:
        return "Debt Risk Spike"
    elif row["bill_due_in_days"] < 3:
        return "Bill Due Soon"
    elif row["savings_progress"] > 0.8 and row["daily_spending"] < 300:
        return "Excellent Budgeting"
    else:
        return "Normal"

df["behavior_context"] = df.apply(classify_behavior, axis=1)


In [19]:
def generate_nudge(profile, context):
    calm_nudges = {
        "Overspending & Under-saving": "Keep calm and track each expense. You're just a tweak away from balance.",
        "Debt Risk Spike": "You've taken a bold step. Let's not let debt win the race.",
        "Bill Due Soon": "You're organized. Just a gentle reminder: bill due soon.",
        "Excellent Budgeting": "Peaceful and powerful — you're managing finances beautifully.",
        "Normal": "You're on track. Stay balanced and mindful."
    }

    aggressive_nudges = {
        "Overspending & Under-saving": "Yo! Time to rein it in. Cash is bleeding fast.",
        "Debt Risk Spike": "Red alert! You're gearing into debt mode.",
        "Bill Due Soon": "Get ahead of the game. Smash those bills now!",
        "Excellent Budgeting": "Legend move. You're optimizing like a pro.",
        "Normal": "Doing good — but can you push for more savings?"
    }

    moderate_nudges = {
        "Overspending & Under-saving": "Looks like spending went wild. Maybe revisit your goals?",
        "Debt Risk Spike": "Debt curve’s rising — maybe pause a moment?",
        "Bill Due Soon": "Bill ahead — no stress if you're prepped!",
        "Excellent Budgeting": "You're setting a good rhythm — keep it up!",
        "Normal": "Things are balanced — maybe think of a saving boost?"
    }

    if profile == "calm":
        return calm_nudges[context]
    elif profile == "aggressive":
        return aggressive_nudges[context]
    else:
        return moderate_nudges[context]

df["nudge"] = df.apply(lambda row: generate_nudge(row["profile"], row["behavior_context"]), axis=1)


In [20]:
def real_time_check(new_point_dict):
    df_new = pd.DataFrame([new_point_dict])
    df_new["profile_encoded"] = profile_enc.transform(df_new["profile"])
    X_new_scaled = scaler.transform(df_new[features])
    
    anomaly = model.predict(X_new_scaled)[0]
    anomaly = 1 if anomaly == -1 else 0

    context = classify_behavior(df_new.iloc[0])
    nudge = generate_nudge(df_new.iloc[0]["profile"], context)
    
    return {
        "anomaly_detected": bool(anomaly),
        "behavior_context": context,
        "nudge_message": nudge
    }


In [21]:
sample_input = {
    "daily_spending": 950,
    "savings_progress": 0.2,
    "debt_ratio": 0.8,
    "bill_due_in_days": 2,
    "profile": "aggressive"
}

real_time_check(sample_input)


{'anomaly_detected': True,
 'behavior_context': 'Overspending & Under-saving',
 'nudge_message': 'Yo! Time to rein it in. Cash is bleeding fast.'}